### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [46]:
#Note field header
#Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
pymoli_purchase_data = "/users/brant/Documents/DABC/pandas-resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data_file = pd.read_csv(pymoli_purchase_data)
purchase_data_file.head()
                    

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [47]:
uniplayercount = purchase_data_file.SN.nunique()
uniplayerdf = pd.DataFrame({"Unique Player Count":[uniplayercount]})
# print player count with narrative.
print("")
print(str(uniplayercount) + ' Players made an in-game purchase.')
print("")
uniplayerdf.style.hide_index()



576 Players made an in-game purchase.



Unique Player Count
576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [48]:
# create data for unique items, average sale price, totals sales, etc. 
countunique = purchase_data_file['Item ID'].nunique()
avgprice = '${:.2f}'.format(purchase_data_file['Price'].mean())
countpurchases = purchase_data_file['Purchase ID'].count()
totalsales = '${:,.2f}'.format(purchase_data_file['Price'].sum())

# print narrative of the data
print("")
print(str(countunique) + " unique items were sold in a total of " + str(countpurchases) + " transactions.")
print("")
print("Sales totalled " + str(totalsales) + " with an Average Sale Price of " + str(avgprice))
print("")

# Create summary dataframe to display data
purchasestats = pd.DataFrame({"Count of Unique Items":[countunique], "Count Purchases":[countpurchases],
                              "Avg Purchase Price":[avgprice], "Total Sales":[totalsales]},index=['Stats'])

# Transpose DataFame for improved display
purchasestats_t = purchasestats.transpose()

# Display the summary data
purchasestats_t




183 unique items were sold in a total of 780 transactions.

Sales totalled $2,379.77 with an Average Sale Price of $3.05



,Stats
Count of Unique Items,183
Count Purchases,780
Avg Purchase Price,$3.05
Total Sales,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [49]:
#group data by gender and count unique SN
gender_data = purchase_data_file.groupby('Gender')['SN'].nunique()

otherperc = '{:.2f}%'.format((gender_data.iloc[2]/uniplayercount)*100)
femaleperc = '{:.2f}%'.format((gender_data.iloc[0]/uniplayercount)*100)
maleperc = '{:.2f}%'.format((gender_data.iloc[1]/uniplayercount)*100)

# narrative of gender demographics
print("")
print("The majority of players are Male, " + str(maleperc) + " with a smaller number of Female players, " + 
      str(femaleperc) + ".")
print("")
# Create summary DataFrame
gender_summary = pd.DataFrame([
    {"Gender":"Other/ND","Count": gender_data.iloc[2],"Percent": otherperc}, 
    {"Gender":"Female","Count": gender_data.iloc[0],"Percent": femaleperc},
    {"Gender":"Male","Count": gender_data.iloc[1],"Percent": maleperc},
    ]).set_index('Gender').reset_index()
# gender_summary.set_index('Gender').reset_index()
gender_summary.sort_values("Count", ascending=False).style.hide_index()



The majority of players are Male, 84.03% with a smaller number of Female players, 14.06%.



Gender,Count,Percent
Male,484,84.03%
Female,81,14.06%
Other/ND,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [50]:
# Create DataFrame for purchase count by gender
genpurchasecount = purchase_data_file.groupby('Gender')['Price'].count().reset_index(name='Count')


# Create DataFrame for average price by gender 
genavgprice = purchase_data_file.groupby('Gender')['Price'].mean().reset_index(name='Avg Price')


# Create DataFrame for total purchase by gender
gentotalpurch = purchase_data_file.groupby('Gender')['Price'].sum().reset_index(name= 'Total Purchase')


# Create DataFrame for total purchase by person average by gender
gentotalpurchavgperperson = purchase_data_file.groupby('Gender')['Price'].sum()/purchase_data_file.groupby('Gender')['SN'].nunique()

# add default index
gentotalpurchavgperperson = gentotalpurchavgperperson.reset_index()
gentotalpurchavgperperson.columns = ['Gender', 'Avg Total Purchase Per Person']


# merge dataframes to prepare summary dataframe
genderstats = pd.merge(genpurchasecount, gentotalpurch, on='Gender')
genderstats = pd.merge(genderstats, genavgprice, on='Gender')
genderstats = pd.merge(genderstats, gentotalpurchavgperperson, on='Gender')


# prepare data for display - update column names, apply create format dictionary
genderstats.columns = ['Gender','Purchase Count', 'Total Purchase Value', 'Average Purchase Price', 'Avg Total Purchase per Person'] 

formatstyle = {'Average Purchase Price': '${:,.2f}','Total Purchase Value': '${:,.2f}','Avg Total Purchase per Person': '${:,.2f}'}


# display stats
genderstats.sort_values("Purchase Count", ascending=False).style.format(formatstyle).hide_index()


Gender,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Male,652,"$1,967.64",$3.02,$4.07
Female,113,$361.94,$3.20,$4.47
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [51]:
# Create the bins for age data   
bins = [0, 9, 14, 19, 24, 29, 34, 39, 150]
binlables = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# add age bins to dataframe
purchase_data_file["Age Groups"] = pd.cut(purchase_data_file["Age"], bins, labels=binlables)

# create and organize dataframe for analysis data
agebins = purchase_data_file.groupby("Age Groups").count()
agebins = agebins[["Age"]]
agebins = agebins.rename(columns={"Age": "Total Count"})

# calculate age bin stats
agebintotals = purchase_data_file["Age Groups"].value_counts()
agebinperc = agebins["Total Count"] / uniplayercount * 100

agedemo = pd.DataFrame({"Total Count": agebintotals, "Percent of Players": agebinperc})

agedemo["Percent of Players"] = agedemo["Percent of Players"].map("{:,.2f}%".format)
agedemo = agedemo.sort_index()

print("")

agedemo


,Total Count,Percent of Players
<10,23,3.99%
10-14,28,4.86%
15-19,136,23.61%
20-24,365,63.37%
25-29,101,17.53%
30-34,73,12.67%
35-39,41,7.12%
40+,13,2.26%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [52]:
# Create data for purchsae analysis by age
agebinpurch = purchase_data_file.groupby("Age Groups").count()

# organize dataframe
agebinpurch = agebinpurch[["Age"]]
agebinpurch = agebinpurch.rename(columns={"Age": "Purchase Count"})

# create data for purchase analysis by age
ageavgprice = purchase_data_file.groupby("Age Groups").mean()

#organize dataframe
ageavgprice = ageavgprice[["Price"]]
ageavgprice = ageavgprice.rename(columns={"Price": "Average Purchase Price"})

# merge dataframes 
dfphase1 = pd.concat([ageavgprice, agebinpurch], axis=1)

# create total purchase data by age
agetotpurch = purchase_data_file.groupby("Age Groups").sum()
agetotpurch = agetotpurch[["Price"]]
agetotpurch = agetotpurch.rename(columns={"Price": "Total Purchase Value"})

# merge dataframes
dfphase2 = pd.concat([dfphase1, agetotpurch], axis=1)

dfphase2["Average Purchase Total per Person"] = dfphase2["Total Purchase Value"] / dfphase2["Purchase Count"]
agepurchstats = dfphase2[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Average Purchase Total per Person"]]

#prepare summary data for display
agepurchstats["Average Purchase Price"] = agepurch["Average Purchase Price"].map("${:,.2f}".format)
agepurchstats["Total Purchase Value"] = agepurch["Total Purchase Value"].map("${:,.2f}".format)
agepurchstats["Average Purchase Total per Person"] = agepurch["Average Purchase Total per Person"].map("${:,.2f}".format)


agepurchstats

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Age Groups,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,"$1,114.06",$3.05
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [53]:
playergroup = purchase_data_file.groupby(["SN"])

# Collect and organize purchase data by player
topfive = playergroup[["Price"]].sum().nlargest(5,"Price")
topfive = topfive.rename(columns={"Price": "Total Purchase Value"})

purchases = playergroup[["Price"]].count()
purchases = purchases.rename(columns={"Price": "Purchase Count"})
# merge the dataframes
dfphase1 = topfive.join(purchases)

avgplayerpurch = playergroup[["Price"]].mean()
avgplayerpurch = avgplayerpurch.rename(columns={"Price": "Average Purchase Price"})

# merge the dataframes
dfphase2 = dfphase1.join(avgplayerpurch)
topfive = dfphase2[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

topfive["Average Purchase Price"] = topfive["Average Purchase Price"].map("${:,.2f}".format)
topfive["Total Purchase Value"] = topfive["Total Purchase Value"].map("${:,.2f}".format)

topfive

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [54]:
# group by Item ID and Item Name
purchaseitems = purchase_data_file.groupby(["Item ID","Item Name"])

topitems = purchaseitems[["Price"]].count().nlargest(5,"Price")
topitems = topitems.rename(columns={"Price": "Purchase Count"})


itemprice = purchaseitems[["Price"]].mean()
itemprice = itemprice.rename(columns={"Price": "Item Price"})

dfphase1 = topitems.join(itemprice)

itemtotal = purchaseitems[["Price"]].sum()
itemtotal = itemtotal.rename(columns={"Price": "Total Purchase Value"})
topitems = dfphase1.join(itemtotal)

topitems["Item Price"] = topitems["Item Price"].map("${:,.2f}".format)
topitems["Total Purchase Value"] = topitems["Total Purchase Value"].map("${:,.2f}".format)

topitems

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
145,Fiery Glass Crusader,9,$4.58,$41.22
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [55]:
# same basic code as the previous sections
topsales = purchaseitems[["Price"]].sum().nlargest(5,"Price")
topsales = topsales.rename(columns={"Price": "Total Purchase Value"})

itempurch = purchaseitems[["Price"]].count()
itempurch = itempurch.rename(columns={"Price": "Purchase Count"})

dfphase1 = topsales.join(itempurch)

itemprice = purchaseitems[["Price"]].mean()
itemprice = itemprice.rename(columns={"Price": "Item Price"})

dfphase2 = dfphase1.join(itemprice)

topsales = dfphase2[["Purchase Count", "Item Price", "Total Purchase Value"]]

topsales["Item Price"] = topsales["Item Price"].map("${:,.2f}".format)
topsales["Total Purchase Value"] = topsales["Total Purchase Value"].map("${:,.2f}".format)

topsales

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
